<a href="https://colab.research.google.com/github/yeesem/Deep_Learning/blob/main/Basic_Mirrored_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
# Import TensorFlow and TensorFlow Datasets

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

# Load and Preprocess Data

In [2]:
# Load the dataset we'll use for this lab
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir='./data')

mnist_train, mnist_test = datasets['train'], datasets['test']

Dataset mnist downloaded and prepared to ./data/mnist/3.0.1. Subsequent calls will reuse this data.


In [11]:
# Define the strategy to use and print the numbner of devices used
strategy = tf.distribute.MirroredStrategy()
print("Number of devices : {}".format(strategy.num_replicas_in_sync))

Number of devices : 1


In [12]:
# Get the number of examples in the train and test sets
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
# Use for Mirrored Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# Use for No Strategy
# BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 1

In [15]:
def scale(image,label):
  image = tf.cast(image,tf.float32)
  image /= 255.0

  return image,label

In [17]:
# Set up the train and eval dataset
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

# Model Training

In [18]:
# Use for Mirrored Strategy -- comment out `with strategy.scope():` and deindent for no strategy
with strategy.scope():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
    ])

In [19]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [20]:
model.fit(train_dataset, epochs=12)

Epoch 1/12
938/938 [==============================] - 38s 38ms/step - loss: 0.1930 - accuracy: 0.9422
Epoch 2/12
938/938 [==============================] - 28s 30ms/step - loss: 0.0645 - accuracy: 0.9808
Epoch 3/12
938/938 [==============================] - 26s 28ms/step - loss: 0.0456 - accuracy: 0.9859
Epoch 4/12
938/938 [==============================] - 26s 28ms/step - loss: 0.0336 - accuracy: 0.9897
Epoch 5/12
938/938 [==============================] - 26s 28ms/step - loss: 0.0263 - accuracy: 0.9919
Epoch 6/12
938/938 [==============================] - 25s 27ms/step - loss: 0.0196 - accuracy: 0.9937
Epoch 7/12
938/938 [==============================] - 25s 27ms/step - loss: 0.0151 - accuracy: 0.9954
Epoch 8/12
938/938 [==============================] - 28s 30ms/step - loss: 0.0109 - accuracy: 0.9966
Epoch 9/12
938/938 [==============================] - 26s 28ms/step - loss: 0.0090 - accuracy: 0.9973
Epoch 10/12
938/938 [==============================] - 26s 28ms/step - loss: 0.007